In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib
import sklearn
import statsmodels

In [35]:
df = pd.read_csv('train_IxoE5JN.csv', parse_dates = ['datetime'], index_col = 'datetime')
df.drop("row_id",axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94992 entries, 2008-03-01 00:00:00 to 2018-12-31 23:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   energy  93092 non-null  float64
dtypes: float64(1)
memory usage: 1.4 MB


In [36]:
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df

df = create_features(df)
df

,energy,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
datetime,,,,,,,,,
2008-03-01 00:00:00,1259.985563,0,5,1,3,2008,61,1,9
2008-03-01 01:00:00,1095.541500,1,5,1,3,2008,61,1,9
2008-03-01 02:00:00,1056.247500,2,5,1,3,2008,61,1,9
2008-03-01 03:00:00,1034.742000,3,5,1,3,2008,61,1,9
2008-03-01 04:00:00,1026.334500,4,5,1,3,2008,61,1,9
...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,2528.446800,19,0,4,12,2018,365,31,1
2018-12-31 20:00:00,2461.883600,20,0,4,12,2018,365,31,1
2018-12-31 21:00:00,2365.994400,21,0,4,12,2018,365,31,1


In [37]:
def add_lags(df):
    target_map = df['energy'].to_dict()
    df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
    return df
df = add_lags(df)
df

,energy,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,lag1,lag2,lag3
datetime,,,,,,,,,,,,
2008-03-01 00:00:00,1259.985563,0,5,1,3,2008,61,1,9,NaN,NaN,NaN
2008-03-01 01:00:00,1095.541500,1,5,1,3,2008,61,1,9,NaN,NaN,NaN
2008-03-01 02:00:00,1056.247500,2,5,1,3,2008,61,1,9,NaN,NaN,NaN
2008-03-01 03:00:00,1034.742000,3,5,1,3,2008,61,1,9,NaN,NaN,NaN
2008-03-01 04:00:00,1026.334500,4,5,1,3,2008,61,1,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,2528.446800,19,0,4,12,2018,365,31,1,2142.5796,1957.0518,2258.8670
2018-12-31 20:00:00,2461.883600,20,0,4,12,2018,365,31,1,2102.3100,1901.7234,2213.5308
2018-12-31 21:00:00,2365.994400,21,0,4,12,2018,365,31,1,2049.0120,1867.3335,2158.7852


In [38]:
df['lag1'].isnull().sum()

10463

In [39]:
def add_lags(df):
    target_map = df['energy'].to_dict()
#     df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
#     df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
    df['lag4'] = (df.index - pd.Timedelta('1456 days')).map(target_map)
    df['lag5'] = (df.index - pd.Timedelta('1820 days')).map(target_map)
    
#     df['lag6'] = (df.index - pd.Timedelta('182 days')).map(target_map)
#     df['lag7'] = (df.index - pd.Timedelta('546 days')).map(target_map)
    df['lag8'] = (df.index - pd.Timedelta('910 days')).map(target_map)
    df['lag9'] = (df.index - pd.Timedelta('1274 days')).map(target_map)
    df['lag10'] = (df.index - pd.Timedelta('1638 days')).map(target_map)
#     df['lag11'] = (df.index - pd.Timedelta('2002 days')).map(target_map)
#     df['lag12'] = (df.index - pd.Timedelta('2184 days')).map(target_map)
#     df['lag13'] = (df.index - pd.Timedelta('2366 days')).map(target_map)
    return df

In [40]:
def add_lags(df):
    target_map = df['energy'].to_dict()
#     df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
    df['lag4'] = (df.index - pd.Timedelta('1456 days')).map(target_map)
    df['lag5'] = (df.index - pd.Timedelta('1820 days')).map(target_map)
#     df['lag6'] = (df.index - pd.Timedelta('2184 days')).map(target_map)
#     df['lag6'] = (df.index - pd.Timedelta('90 days')).map(target_map)
    return df

In [41]:
df = pd.read_csv('train_IxoE5JN.csv', parse_dates = ['datetime'], index_col = 'datetime')
df.drop("row_id",axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94992 entries, 2008-03-01 00:00:00 to 2018-12-31 23:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   energy  93092 non-null  float64
dtypes: float64(1)
memory usage: 1.4 MB


In [42]:
import xgboost as xgb

In [43]:
df = add_lags(df)
df=df.interpolate()
df = create_features(df)
df

,energy,lag2,lag3,lag4,lag5,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
datetime,,,,,,,,,,,,,
2008-03-01 00:00:00,1259.985563,NaN,NaN,NaN,NaN,0,5,1,3,2008,61,1,9
2008-03-01 01:00:00,1095.541500,NaN,NaN,NaN,NaN,1,5,1,3,2008,61,1,9
2008-03-01 02:00:00,1056.247500,NaN,NaN,NaN,NaN,2,5,1,3,2008,61,1,9
2008-03-01 03:00:00,1034.742000,NaN,NaN,NaN,NaN,3,5,1,3,2008,61,1,9
2008-03-01 04:00:00,1026.334500,NaN,NaN,NaN,NaN,4,5,1,3,2008,61,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,2528.446800,1957.0518,2258.8670,2406.5944,1958.3508,19,0,4,12,2018,365,31,1
2018-12-31 20:00:00,2461.883600,1901.7234,2213.5308,2345.1212,1921.3656,20,0,4,12,2018,365,31,1
2018-12-31 21:00:00,2365.994400,1867.3335,2158.7852,2306.8100,1864.5928,21,0,4,12,2018,365,31,1


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94992 entries, 2008-03-01 00:00:00 to 2018-12-31 23:00:00
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   energy      94992 non-null  float64
 1   lag2        77520 non-null  float64
 2   lag3        68784 non-null  float64
 3   lag4        60048 non-null  float64
 4   lag5        51312 non-null  float64
 5   hour        94992 non-null  int64  
 6   dayofweek   94992 non-null  int64  
 7   quarter     94992 non-null  int64  
 8   month       94992 non-null  int64  
 9   year        94992 non-null  int64  
 10  dayofyear   94992 non-null  int64  
 11  dayofmonth  94992 non-null  int64  
 12  weekofyear  94992 non-null  UInt32 
dtypes: UInt32(1), float64(5), int64(7)
memory usage: 9.9 MB


In [45]:
df.isnull().sum()

energy            0
lag2          17472
lag3          26208
lag4          34944
lag5          43680
hour              0
dayofweek         0
quarter           0
month             0
year              0
dayofyear         0
dayofmonth        0
weekofyear        0
dtype: int64

In [46]:
# df.drop('lag3',inplace=True,axis=1)

In [47]:
df=df.dropna()

In [176]:

FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year',
            'lag1','lag2','lag3']
FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year',
            'lag1','lag2']
FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year',
            'lag3','lag4','lag5']
FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year'
            ,'lag2','lag3','lag4','lag5']
TARGET = 'energy'

X_all = df[FEATURES]
y_all = df[TARGET]

# reg = xgb.XGBRegressor(base_score=0.5,
#                        booster='gbtree',    
#                        n_estimators=500,
#                        objective='reg:linear',
#                        max_depth=3,
#                        learning_rate=0.01)


# reg = xgb.XGBRegressor(base_score=0.5,
#                        booster='gbtree',    
#                        n_estimators=1000,
#                        objective='reg:linear',
#                        max_depth=3,
#                        learning_rate=0.008)


reg = xgb.XGBRegressor(base_score=0.5,
                       booster='gbtree',    
                       n_estimators=1500,
                       objective='reg:linear',
                       max_depth=3,
                       learning_rate=0.005,reg_alpha=0.05)


# reg = xgb.XGBRegressor(base_score=0.5,
#                        booster='gbtree',    
#                        n_estimators=1500,
#                        objective='reg:linear',
#                        max_depth=3,
#                        learning_rate=0.005,reg_alpha=0.05,subsample=0.8)


# reg = catboost.CatBoostRegressor(loss_function='RMSE',
#                        max_depth=3,
#                        learning_rate=0.03,iterations=2000)





reg.fit(X_all, y_all,
        eval_set=[(X_all, y_all)],
        verbose=100)

[23:59:28] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1916.97365
[100]	validation_0-rmse:1173.73260
[200]	validation_0-rmse:729.46263
[300]	validation_0-rmse:468.96745
[400]	validation_0-rmse:322.25389
[500]	validation_0-rmse:244.76812
[600]	validation_0-rmse:207.13030
[700]	validation_0-rmse:189.10713
[800]	validation_0-rmse:180.08876
[900]	validation_0-rmse:175.06378
[1000]	validation_0-rmse:171.75261
[1100]	validation_0-rmse:169.31087
[1200]	validation_0-rmse:167.28060
[1300]	validation_0-rmse:165.60089
[1400]	validation_0-rmse:163.94837
[1499]	validation_0-rmse:162.48731


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.005, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=3, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1500, n_jobs=0,
             num_parallel_tree=1, objective='reg:linear', predictor='auto', ...)

In [177]:
future = pd.date_range('01-01-2019 00:00','31-12-2021 23:00', freq='1h')
future_df = pd.DataFrame(index=future)
future_df['isFuture'] = True
df['isFuture'] = False
df_and_future = pd.concat([df, future_df])
df_and_future = create_features(df_and_future)
df_and_future = add_lags(df_and_future)

C:\Users\cools\AppData\Local\Temp\ipykernel_228\2121270473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isFuture'] = False


In [178]:
future_w_features = df_and_future.query('isFuture').copy()

In [179]:
future_w_features['pred'] = reg.predict(future_w_features[FEATURES])

In [180]:
df1 = pd.read_csv('test_WudNWDM.csv',parse_dates=True,index_col='datetime')
# df1.drop('row_id',axis=1,inplace=True)
test=df1.copy()
df1.head()

,row_id
datetime,
2019-01-01 00:00:00,94993
2019-01-01 01:00:00,94994
2019-01-01 02:00:00,94995
2019-01-01 03:00:00,94996
2019-01-01 04:00:00,94997


In [181]:
df1['energy'] = future_w_features['pred']
df1

,row_id,energy
datetime,,
2019-01-01 00:00:00,94993,1918.500122
2019-01-01 01:00:00,94994,1874.026245
2019-01-01 02:00:00,94995,1751.831543
2019-01-01 03:00:00,94996,1723.062988
2019-01-01 04:00:00,94997,1702.992310
...,...,...
2021-12-31 19:00:00,121292,2090.886719
2021-12-31 20:00:00,121293,2089.596924
2021-12-31 21:00:00,121294,2084.176514


In [182]:
df1.set_index('row_id',inplace=True)
df1

,energy
row_id,
94993,1918.500122
94994,1874.026245
94995,1751.831543
94996,1723.062988
94997,1702.992310
...,...
121292,2090.886719
121293,2089.596924
121294,2084.176514


In [183]:
df1.to_csv("file41.csv")

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib
import sklearn
import statsmodels

df = pd.read_csv('train_IxoE5JN.csv', parse_dates = ['datetime'], index_col = 'datetime')
df.drop("row_id",axis=1,inplace=True)
df.info()

def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df

df = create_features(df)
df

def add_lags(df):
    target_map = df['energy'].to_dict()
    df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
    return df
df = add_lags(df)
df

df['lag1'].isnull().sum()









def add_lags(df):
    target_map = df['energy'].to_dict()
#     df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
#     df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
    df['lag4'] = (df.index - pd.Timedelta('1456 days')).map(target_map)
    df['lag5'] = (df.index - pd.Timedelta('1820 days')).map(target_map)
    
#     df['lag6'] = (df.index - pd.Timedelta('182 days')).map(target_map)
#     df['lag7'] = (df.index - pd.Timedelta('546 days')).map(target_map)
    df['lag8'] = (df.index - pd.Timedelta('910 days')).map(target_map)
    df['lag9'] = (df.index - pd.Timedelta('1274 days')).map(target_map)
    df['lag10'] = (df.index - pd.Timedelta('1638 days')).map(target_map)
#     df['lag11'] = (df.index - pd.Timedelta('2002 days')).map(target_map)
#     df['lag12'] = (df.index - pd.Timedelta('2184 days')).map(target_map)
#     df['lag13'] = (df.index - pd.Timedelta('2366 days')).map(target_map)
    return df

def add_lags(df):
    target_map = df['energy'].to_dict()
#     df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
    df['lag4'] = (df.index - pd.Timedelta('1456 days')).map(target_map)
    df['lag5'] = (df.index - pd.Timedelta('1820 days')).map(target_map)
#     df['lag6'] = (df.index - pd.Timedelta('2184 days')).map(target_map)
#     df['lag6'] = (df.index - pd.Timedelta('90 days')).map(target_map)
    return df





df = pd.read_csv('train_IxoE5JN.csv', parse_dates = ['datetime'], index_col = 'datetime')
df.drop("row_id",axis=1,inplace=True)
df.info()

import xgboost as xgb

df = add_lags(df)
df=df.interpolate(method = 'spline', order = 2)
df = create_features(df)
df

df.info()

df.isnull().sum()

# df.drop('lag3',inplace=True,axis=1)

df=df.dropna()


FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year',
            'lag1','lag2','lag3']
FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year',
            'lag1','lag2']
FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year',
            'lag3','lag4','lag5']
FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year'
            ,'lag2','lag3','lag4','lag5']
TARGET = 'energy'

X_all = df[FEATURES]
y_all = df[TARGET]

# reg = xgb.XGBRegressor(base_score=0.5,
#                        booster='gbtree',    
#                        n_estimators=500,
#                        objective='reg:linear',
#                        max_depth=3,
#                        learning_rate=0.01)


# reg = xgb.XGBRegressor(base_score=0.5,
#                        booster='gbtree',    
#                        n_estimators=1000,
#                        objective='reg:linear',
#                        max_depth=3,
#                        learning_rate=0.008)


reg = xgb.XGBRegressor(base_score=0.5,
                       booster='gbtree',    
                       n_estimators=1500,
                       objective='reg:linear',
                       max_depth=3,
                       learning_rate=0.005)

# reg = catboost.CatBoostRegressor(loss_function='RMSE',
#                        max_depth=3,
#                        learning_rate=0.03,iterations=2000)





reg.fit(X_all, y_all,
        eval_set=[(X_all, y_all)],
        verbose=100)

future = pd.date_range('01-01-2019 00:00','31-12-2021 23:00', freq='1h')
future_df = pd.DataFrame(index=future)
future_df['isFuture'] = True
df['isFuture'] = False
df_and_future = pd.concat([df, future_df])
df_and_future = create_features(df_and_future)
df_and_future = add_lags(df_and_future)

future_w_features = df_and_future.query('isFuture').copy()

future_w_features['pred'] = reg.predict(future_w_features[FEATURES])

df1 = pd.read_csv('test_WudNWDM.csv',parse_dates=True,index_col='datetime')
# df1.drop('row_id',axis=1,inplace=True)
test=df1.copy()
df1.head()

df1['energy'] = future_w_features['pred']
df1

df1.set_index('row_id',inplace=True)
df1

df1.to_csv("file41.csv")

























































